In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import scipy.interpolate
from astropy.io import fits
from matplotlib.colors import SymLogNorm, Normalize
from PIL import Image

In [2]:
original = fits.open('../../2019/12/bgmodel.fits')[0].data
original.shape

(24, 24)

In [3]:
plt.figure(figsize=(10,10))
plt.imshow(original, cmap='Greys_r')
plt.colorbar()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 1D interpolation

In [4]:
def interpolate(y, x, newx):
    return scipy.interpolate.interp1d(x, y, 'cubic')(newx)

In [5]:
def myinterpolate(y, x, newx):
    # https://en.wikiversity.org/wiki/Cubic_Spline_Interpolation
    size = len(x)
    newsize = len(newx)
    result = np.zeros(newsize)
    intervals = []
    
    h = np.zeros(size)
    h[1:] = x[1:] - x[:-1]
    u = np.zeros(size)
    u[:-1] = h[:-1] / (h[:-1] + h[1:])
    l = 1 - u
    l[0] = u[0] = 0
    
    d = np.zeros(size)
    d[0] = d[-1] = 0.
    d[1:-1] = 6 * ((y[2:] - y[1:-1]) / (x[2:] - x[1:-1]) - (y[1:-1] - y[:-2]) / (x[1:-1] - x[0:-2])) / (x[2:] - x[:-2])

    A = np.eye(size) * 2
    for i in range(size-1):
        A[i+1, i] = u[i+1]
        A[i, i+1] = l[i]
    
    AI = np.linalg.inv(A)
    M = AI.dot(d)
    
    for ni, nx in enumerate(newx):
        i = int(nx) + 1
        if i > size-1:
            i = size-1
        result[ni] =   M[i-1] * (x[i] - nx)**3 / 6 * h[i] \
                    + M[i]  * (nx - x[i-1])**3 / 6 * h[i] \
                    + (y[i-1] - (M[i-1]*h[i]**2)/6) * (x[i] - nx) / h[i] \
                    + (y[i] - (M[i] * h[i]**2)/6) * (nx - x[i-1]) / h[i]
        
    return result, intervals

In [6]:
data_y = original[13,0:50]
data_x = np.arange(len(data_y))

In [7]:
new_x = np.linspace(0, len(data_y)-1, 100)

In [8]:
scinew = interpolate(data_y, data_x, new_x)
mynew, intervals = myinterpolate(data_y, data_x, new_x)

In [14]:
plt.close()

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(20,15), sharex=True)
plt.subplots_adjust(hspace=0)
axes.plot(data_x, data_y, marker='o', label='Downsampled', c='orange')
axes.plot(new_x, scinew, marker='x', label='SciPy', c='green')
axes.plot(new_x, mynew, marker='.', label='My', c='blue', linestyle='--')

if False:
    for idx in [0, 1, 5, 6, 9, 10]:
        a, b, c, d = intervals[idx]
        v = np.linspace(-1,2,100)
        l = a * v**3 + b * v**2 + c * v + d
        axes.plot(data_x[idx]+ v, l, c='r', linestyle=':')

axes.legend()

plt.show()

if False:
    # Some consistency tests
    def derivative_at_point(f, p, epsilon=1e-4):
        x = np.linspace(p - epsilon, p+epsilon, 9)
        y = f(x)
        dy = np.gradient(y, x[1]-x[0])
        ddy = np.gradient(dy, x[1]-x[0])
        return dy[4], ddy[4]

    for idx in range(len(intervals) - 1):
        a, b, c, d = intervals[idx]
        lf = lambda v: a * v**3 + b * v**2 + c * v + d
        lv = lf(1)
        ldy, lddy = derivative_at_point(lf, 1)

        a, b, c, d = intervals[idx+1]
        rf = lambda v: a * v**3 + b * v**2 + c * v + d
        rv = rf(0)
        rdy, rddy = derivative_at_point(lambda v: a * v**3 + b * v**2 + c * v + d, 0)

        # Derivative of the right and left piecewises at this point is the same
        if not np.isclose(ldy, rdy):
            print(idx, ldy, rdy)
        print(idx, lddy, rddy)
        # Value of the right and left piecewises at the intersection is the same
        assert np.isclose(lv, rv)
        # And they match with the actual interpolated value
        assert np.isclose(lv, data_y[idx+1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 2D interpolation

In [10]:
cell_size = 16
newsize = original.shape[0]*cell_size, original.shape[1]*cell_size

In [11]:
ox = np.arange(original.shape[0])
oy = np.arange(original.shape[1])
nx = np.arange(newsize[0])/(newsize[0]-1) * (original.shape[0]-1)
ny = np.arange(newsize[1])/(newsize[1]-1) * (original.shape[1]-1)
sample = scipy.interpolate.interp2d(ox, oy, original, kind='cubic')(nx, ny)

In [12]:
img = np.zeros(newsize)
for i in range(original.shape[1]):
    img[:, i*cell_size], _ = myinterpolate(original[:, i], ox, nx)
for i in range(img.shape[0]):
    img[i, :], _ = myinterpolate(img[i, ::cell_size], oy, ny)

In [13]:
fig, axes = plt.subplots(ncols=3, nrows=1, figsize=(22, 10))
axes[0].imshow(img, cmap='Greys_r')
axes[1].imshow(sample, cmap='Greys_r')
d = axes[2].imshow(img-sample)
plt.colorbar(d)
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …